<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/project_configuration/project_setup.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/project_configuration/project_setup.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Project Setup
* This notebok describes how to create and configure a project
* This is the same as creating a new project in the editor and going through all of the steps.

* When a user creates a project with client.create_project() the project is not ready for labeling.
    * An ontology must be set
    * Datasets/Batchh must be attached

In [ ]:
!pip install labelbox[data]

In [ ]:
import labelbox as lb
import uuid

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY=None
client = lb.Client(api_key=API_KEY)

### Create Dataset
* Create dataset and attach data
* More details on attaching data can be found [here](https://github.com/Labelbox/labelbox-python/blob/master/examples/basics/data_rows.ipynb)

In [ ]:
dataset = client.create_dataset(name="project_setup_demo")
## Example image
uploads = []
# Generate data rows
for i in range(1,9):
    uploads.append({
        'row_data':  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key": f"ID-{ uuid.uuid1()}"
    })
data_rows = dataset.create_data_rows(uploads)
data_rows.wait_till_done()
print("ERRORS: " , data_rows.errors)
print("RESULT URL: ", data_rows.result_url)

ERRORS:  None
RESULT URL:  https://storage.labelbox.com/cl3ahv73w1891087qbwzs3edd%2Fdata-row-imports-results%2Fcl9sj9crk0zf307yo0mnm17y4_cl9sj9e1u50gm07upd3xj7vyx.json?Expires=1667050292662&KeyName=labelbox-assets-key-3&Signature=JW912R-7wveaHGyhkdKIBRKEtVw


### Create Or Select an Ontology
* Optionally create an ontology or select from an existing one.
* More details on ontology management can be found [here](https://github.com/Labelbox/labelbox-python/blob/master/examples/basics/ontologies.ipynb)
   

In [ ]:
# Create
ontology_builder = lb.OntologyBuilder(
    tools=[lb.Tool(name="cat", tool=lb.Tool.Type.BBOX)])
ontology = client.create_ontology("project-setup-demo-ontology",
                                  ontology_builder.asdict())
# Select existing ontology
#ontology = client.get_ontology("<ontology_id>")
#ontology = existing_project.ontology()

### Create Project and Setup the Editor
* Setting up a project will add an ontology and will enable labeling to begin
* Attaching dataset(s) will add all data_rows belonging to the dataset to the queue.
* Creating batches will add data rows exported from a dataset

In [ ]:
batch_project = client.create_project(name="batch-test-project", 
                                media_type=lb.MediaType.Image,
                                # Setup quality settings (Benchmarks)
                                auto_audit_percentage=1,
                                auto_audit_number_of_labels=1,
                                queue_mode=lb.QueueMode.Batch)


dataset_project = client.create_project(name="dataset-test-project",
                                description="a description",
                                media_type=lb.MediaType.Image,
                                ## Setup quality settings (Consensus)
                                auto_audit_percentage=0.1,
                                auto_audit_number_of_labels=3,
                                queue_mode=lb.QueueMode.Dataset)

batch_project.setup_editor(ontology)
dataset_project.setup_editor(ontology)


# Add data to your projects 

In [ ]:

####### Connect your dataset 
# Dataset projects do not support batches

dataset_project.datasets.connect(dataset)

######## Create batches 
# Batch projects do not support datasets

# We need the data row ID to create a batch
batch_datarows = [dr.uid for dr in list(dataset.export_data_rows())]

## When creating a batch you can also setup the data rows priority
batch = batch_project.create_batch(
  "batch-demo", # Each batch in a project must have a unique name
  batch_datarows, # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)
print("Batch: ", batch)

Batch:  <Batch {'created_at': datetime.datetime(2022, 10, 28, 13, 32, 56, tzinfo=datetime.timezone.utc), 'name': 'batch-demo', 'size': 8, 'uid': '086cfda0-56c5-11ed-a20f-291accbaa64c', 'updated_at': datetime.datetime(2022, 10, 28, 13, 32, 56, tzinfo=datetime.timezone.utc)}>


### Review

In [ ]:
# Note setup_complete will be None if it fails.
print(batch_project.setup_complete)
print(dataset_project.setup_complete)
print(batch_project.ontology())

print(dataset_project.ontology())
print([ds.name for ds in batch_project.batches()])
print([ds.name for ds in dataset_project.datasets()])

2022-10-28 13:32:45+00:00
2022-10-28 13:32:50+00:00
<Ontology {'classification_schema_count': 0, 'created_at': datetime.datetime(2022, 10, 28, 13, 31, 37, 384000, tzinfo=datetime.timezone.utc), 'description': None, 'name': 'project-setup-demo-ontology', 'normalized': {'tools': [{'schemaNodeId': 'cl9sj9j5c59ig08z586542v8d', 'featureSchemaId': 'cl9sj9j5c59if08z54zqmam0f', 'required': False, 'name': 'cat', 'tool': 'rectangle', 'color': '#ff0000', 'archived': 0, 'classifications': []}], 'classifications': [], 'relationships': []}, 'object_schema_count': 1, 'uid': 'cl9sj9j5059ie08z57041es1r', 'updated_at': datetime.datetime(2022, 10, 28, 13, 31, 37, 384000, tzinfo=datetime.timezone.utc)}>
<Ontology {'classification_schema_count': 0, 'created_at': datetime.datetime(2022, 10, 28, 13, 31, 37, 384000, tzinfo=datetime.timezone.utc), 'description': None, 'name': 'project-setup-demo-ontology', 'normalized': {'tools': [{'schemaNodeId': 'cl9sj9j5c59ig08z586542v8d', 'featureSchemaId': 'cl9sj9j5c59if0

In [ ]:
print(f"https://app.labelbox.com/projects/{batch_project.uid}")
print(f"https://app.labelbox.com/projects/{dataset_project.uid}")

https://app.labelbox.com/projects/cl9sjavsl2oqx07w92abm9zy0
https://app.labelbox.com/projects/cl9sjaw2d2pgq07vddlfsgy6k


# Cleanup

In [ ]:
# batch_project.delete()
# dataset_project.delete()
# dataset.delete()